In [ ]:
# ===========================================================
# ---- These codes are used to run historical tests
# ----- Ekarat Tantawichet
# ===========================================================



# ===========================================================
# ---- IMPORT Packages
# ===========================================================
import finnhub
finnhub_client = finnhub.Client(api_key="<<MASK>>")

import numpy as np
import pandas as pd

from numpy.linalg import inv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import r2_score
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

from sklearn.metrics import mean_squared_error


from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import RepeatedKFold

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")


import xgboost as xgb

from datetime import date, datetime, tzinfo, timedelta
import datetime as dt

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import yfinance as yf

import pypfopt

from pypfopt import risk_models
from pypfopt import plotting

from pypfopt import EfficientFrontier

from pypfopt import risk_models
from pypfopt import plotting

from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel, plotting

from pypfopt import EfficientFrontier, objective_functions

import pickle 

# from google.cloud import storage
# import os

# --------------------------------------------------------------------------- 
from datetime import datetime, timedelta
from textwrap import dedent
import time


####################################################
# DEFINE PYTHON FUNCTIONS
####################################################

# -------- parameters ---------------------
count = 0  # --- this is example of global variable in the def

tickers_pull = ['XOM', 'HON', 'RTX', 'AMZN', 'PEP', 'UNH', 'JNJ', 'V', 'NVDA', 'AAPL', 'MSFT', 'GOOGL']

# ======= initial parameters ============

noofdays_test = 250
# ======= lag 30 days for live runs ============
noofdays_lag_live = 30

# Get today's date
today_real = date.today()

today = today_real
# print("Today is: ", today)

# Yesterday date
yesterday = today - timedelta(days = 1)
# print("Yesterday was: ", yesterday)

# --- market capitalization (December 2023, source Yahoo Finance)----
mcaps = {'XOM': 398158000000,'HON': 132107000000,'RTX': 117750000000,'AMZN': 1508000000000,'PEP': 230729000000,'UNH': 502863000000,'JNJ': 373273000000, 'V': 527197000000,'NVDA': 1152000000000,'AAPL': 3004000000000,'MSFT': 2760000000000,'GOOGL': 1676000000000}


In [ ]:
# ----- regular functions ----------
# ----- date format --------------
def convert_date(x):
    func_date = dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d')
    return func_date


def convert_date_time(x):
    func_date_time = dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')
    return func_date_time

# ----- definition of confidence level --------------
# ------- confidence = 1.00 (high) if the prediction and sentiment agree -------
# ------- confidence = 0.00 (low) if the prediction and sentiment disagree -----
# ------- confidence = 0.50 (medium) if the sentiment is neutral -----

def categorise(row):
    if row['pred'] > 0 and row['sentiment'] ==1:
        return 1
    elif row['pred'] < 0 and row['sentiment'] ==-1:
        return 1
    elif row['pred'] > 0 and row['sentiment'] ==-1:
        return 0
    elif row['pred'] < 0 and row['sentiment'] ==1:
        return 0
    return 0.5

In [ ]:
# ------ dates to be tested -----------------
date_list_x2 = [['2023-10-4', '10/3/2023', '10/2/2023'],
['2023-10-5', '10/4/2023', '10/3/2023'],
['2023-10-6', '10/5/2023', '10/4/2023'],
['2023-10-9', '10/6/2023', '10/5/2023'],
['2023-10-10', '10/9/2023', '10/6/2023'],
['2023-10-11', '10/10/2023', '10/9/2023'],
['2023-10-12', '10/11/2023', '10/10/2023'],
['2023-10-13', '10/12/2023', '10/11/2023'],
['2023-10-16', '10/13/2023', '10/12/2023'],
['2023-10-17', '10/16/2023', '10/13/2023'],
['2023-10-18', '10/17/2023', '10/16/2023'],
['2023-10-19', '10/18/2023', '10/17/2023'],
['2023-10-20', '10/19/2023', '10/18/2023'],
['2023-10-23', '10/20/2023', '10/19/2023'],
['2023-10-24', '10/23/2023', '10/20/2023'],
['2023-10-25', '10/24/2023', '10/23/2023'],
['2023-10-26', '10/25/2023', '10/24/2023'],
['2023-10-27', '10/26/2023', '10/25/2023'],
['2023-10-30', '10/27/2023', '10/26/2023'],
['2023-10-31', '10/30/2023', '10/27/2023'],
['2023-11-1', '10/31/2023', '10/30/2023'],
['2023-11-2', '11/1/2023', '10/31/2023'],
['2023-11-3', '11/2/2023', '11/1/2023'],
['2023-11-6', '11/3/2023', '11/2/2023'],
['2023-11-7', '11/6/2023', '11/3/2023'],
['2023-11-8', '11/7/2023', '11/6/2023'],
['2023-11-9', '11/8/2023', '11/7/2023'],
['2023-11-10', '11/9/2023', '11/8/2023'],
['2023-11-13', '11/10/2023', '11/9/2023'],
['2023-11-14', '11/13/2023', '11/10/2023'],
['2023-11-15', '11/14/2023', '11/13/2023'],
['2023-11-16', '11/15/2023', '11/14/2023'],
['2023-11-17', '11/16/2023', '11/15/2023'],
['2023-11-20', '11/17/2023', '11/16/2023'],
['2023-11-21', '11/20/2023', '11/17/2023'],
['2023-11-22', '11/21/2023', '11/20/2023'],
['2023-11-24', '11/22/2023', '11/21/2023'],
['2023-11-27', '11/24/2023', '11/22/2023'],
['2023-11-28', '11/27/2023', '11/24/2023'],
['2023-11-29', '11/28/2023', '11/27/2023'],
['2023-11-30', '11/29/2023', '11/28/2023'],
['2023-12-1', '11/30/2023', '11/29/2023'],
['2023-12-4', '12/1/2023', '11/30/2023'],
['2023-12-5', '12/4/2023', '12/1/2023'],
['2023-12-6', '12/5/2023', '12/4/2023'],
['2023-12-7', '12/6/2023', '12/5/2023'],
['2023-12-8', '12/7/2023', '12/6/2023'],
['2023-12-11', '12/8/2023', '12/7/2023'],
['2023-12-12', '12/11/2023', '12/8/2023'],
['2023-12-13', '12/12/2023', '12/11/2023'],
['2023-12-14', '12/13/2023', '12/12/2023']]

In [ ]:
# -------- step 1 (tp_1): pull new data and calculate returns etc.
def step1_pull_stock_data(pull_end_date):
    tickers = tickers_pull
    # ohlc = yf.download(tickers, period="60mo") # 60 months start="12/11/18"
    ohlc = yf.download(tickers, start="2018-12-11", end=pull_end_date)
    prices = ohlc["Adj Close"]
    # prices
    # market_prices_BL_raw = yf.download("SPY", period="60mo")["Adj Close"]
    returns_full = prices.pct_change()
    returns= returns_full.dropna()
    # returns
    # returns_full =  returns_full.reset_index()
    # df = returns        
    market_prices_BL_pull = yf.download("SPY", period="60mo")["Adj Close"]
    market_prices_BL_raw = market_prices_BL_pull
    returns.to_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/df.csv', index ='Date')
    prices.to_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/prices.csv', index ='Date')
    market_prices_BL_raw.to_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/market_prices_BL_raw.csv', index='Date') 

In [ ]:
# -------- step 2 (tp_2): fit XGBoost and then make prediction based on new data and calculate returns etc. -----
def step2_ml_xg():
    # ----- XGBoost Parameters ------------
    xg_max_depth=50
    xg_learning_rate=0.8
    xg_reg_lambda=8
    xg_subsample=0.4
    xg_grow_policy="lossguide"    
    # ----- ONE DAY PREDICTION:: LIVE  ------------    
    df = pd.read_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/df.csv', index_col='Date')    
    # training_prices_x = {}
    # training_prices_y = {}
    live_prices_x = {}
    live_prices_y = {}
    live_pred_x = {}
    pred_live_30 = df.tail(noofdays_lag_live)
    # live_pred_x = {}
    for col in df.columns:
        company_live = df[col].to_numpy()
        company_live_30 = pred_live_30[col].to_numpy()
        company_live_x = [company_live[i:i+15] for i in range(len(company_live)-15)]
        company_live_y = [company_live[i+1] for i in range(14,len(company_live)-1)]
        # company_live_pred_x = [company_live[i:i+30+1] for i in range(len(company_live)-30+1)]
        company_live_pred_x= [company_live_30[i:i+15] for i in range(len(company_live)-15)]
        live_prices_x[col] = company_live_x
        live_prices_y[col] = company_live_y
        live_pred_x[col] = [company_live_pred_x[0]]
        # live_pred_x
    # ----- ONE DAY PREDICTION LIVE ------------
    next_day_preds ={}
    for col in df.columns:
        bst = xgb.XGBRegressor(max_depth=xg_max_depth, learning_rate=xg_learning_rate,
                               reg_lambda=xg_reg_lambda, subsample=xg_subsample, grow_policy=xg_grow_policy)
        bst = bst.fit(live_prices_x[col],live_prices_y[col])
        next_day_preds[col] = bst.predict(live_pred_x[col])
    # next_day_preds    
    next_day_preds_df = pd.DataFrame.from_dict(next_day_preds)
    next_day_preds_df.index = ['pred']*1
    # next_day_preds_df
    next_day_preds_df.to_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/next_day_preds_df.csv', index=True)
    with open('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/saved_dictionary.pkl', 'wb') as f_next_day_preds:
        pickle.dump(next_day_preds, f_next_day_preds)

In [ ]:
# -------- step 3 (tp_3): pull news feed and estimate sentiment for each news -----
def step3_news_feed(yesterday_date):    
    stock_list = tickers_pull
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model_finbert = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
    sentiment_list =[]
    for stock in stock_list:

        news_series = finnhub_client.company_news(stock, _from=yesterday_date, to=yesterday_date)
        if news_series == []:
            print("stock news "+stock+" is empty")
            sentiment_stock  = 0
        else:
            news_series_df = pd.DataFrame.from_dict(news_series)
            result_date = []
            result_news = []
            for index, row in news_series_df.iterrows():
                result_news.append(row['headline'])
                result_date.append(convert_date(row['datetime']))
            
            result_date
            inputs = tokenizer(result_news, padding = True, truncation = True, return_tensors='pt')
            outputs = model_finbert(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            
            pred_arr = predictions.detach().cpu().numpy()
            result_date_df = pd.DataFrame({'date' : result_date}, columns=['date'])

            result_news_df = pd.DataFrame(data = pred_arr, columns = ["Positive", "Negative", "Neutral"])

            result_combined = pd.concat([result_date_df, result_news_df], axis=1)

            result_daily_nodate = result_combined.drop(columns=['date'])
            result_daily = result_daily_nodate.mean()

            conditions = [(result_daily['Positive'] > result_daily['Negative']) & (result_daily['Positive'] > result_daily['Neutral']),
            (result_daily['Negative'] > result_daily['Positive']) & (result_daily['Negative'] > result_daily['Neutral']),
            (result_daily['Neutral'] > result_daily['Positive']) & (result_daily['Neutral'] > result_daily['Positive'])]
            sentiment_values = [1, -1, 0]
            result_daily[stock] = np.select(conditions, sentiment_values)
            sentiment_stock  = np.ndarray.item(np.array([result_daily[stock]]))
        sentiment_list.append(sentiment_stock)

    sentiment_list_row =[sentiment_list]
    sentiment_daily = pd.DataFrame(sentiment_list_row, columns=stock_list, index=['sentiment']*1)
    print(tickers_pull)
    print(sentiment_list)
    sentiment_daily.to_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/sentiment_daily.csv', index=True)

In [ ]:
# -------- step 4 (tp_4): run Black Litterman using pypfopt API to get stock holding (portfolio) recommendation -----    
# -------- and then generate previous days performance (portfolio return, standard deviation, and Sharpe-ratio ------  

def step4_bl_weight():  

    prices_BL = pd.read_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/prices.csv', index_col='Date')    
    df_market_csv_read = pd.read_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/market_prices_BL_raw.csv', index_col='Date')
    market_prices_BL_raw=df_market_csv_read[df_market_csv_read.columns[0]]    
    next_day_preds_df = pd.read_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/next_day_preds_df.csv', index_col=['Unnamed: 0'])
    sentiment_daily = pd.read_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/sentiment_daily.csv', index_col=['Unnamed: 0'])    
    with open('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/saved_dictionary.pkl', 'rb') as f_next_day_preds: next_day_preds = pickle.load(f_next_day_preds)    

    next_day_preds_df_T = next_day_preds_df.T

    sentiment_daily_T = sentiment_daily.T

    combine_pred_sentiment = next_day_preds_df_T.join(sentiment_daily_T)
    # combine_pred_sentiment
    combine_pred_sentiment['confidence'] = combine_pred_sentiment.apply(lambda row: categorise(row), axis=1)
    confidence = combine_pred_sentiment['confidence'].T

    market_prices_BL = market_prices_BL_raw
    viewdict_ML2_GBReg = next_day_preds
    confidences_ML2_GBReg = confidence
    S_BL = risk_models.CovarianceShrinkage(prices_BL).ledoit_wolf()
    delta = black_litterman.market_implied_risk_aversion(market_prices_BL)
    # delta
    market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S_BL)
    # market_prior
    bl_ML2_GBReg = BlackLittermanModel(S_BL, pi=market_prior, absolute_views=viewdict_ML2_GBReg, omega="idzorek", view_confidences=confidences_ML2_GBReg)
    bl = bl_ML2_GBReg
    omega_ML2_GBReg = bl.omega
    # We are using the shortcut to automatically compute market-implied prior
    bl_ML2_GBReg = BlackLittermanModel(S_BL, pi="market", market_caps=mcaps, risk_aversion=delta,
    absolute_views=viewdict_ML2_GBReg, omega=omega_ML2_GBReg)
    # Posterior estimate of returns
    ret_bl_ML2_GBReg = bl.bl_returns()
    # ret_bl_ML2_GBReg
    rets_df_ML2_GBReg = pd.DataFrame([market_prior, ret_bl_ML2_GBReg, pd.Series(viewdict_ML2_GBReg)],
     index=["Prior", "Posterior", "Views"]).T
    # rets_df_ML2_GBReg
    # rets_df_ML2_GBReg.plot.bar(figsize=(12,8));
    S_bl_ML2_GBReg = bl_ML2_GBReg.bl_cov()
    # plotting.plot_covariance(S_bl_ML2_GBReg);
    ef = EfficientFrontier(ret_bl_ML2_GBReg, S_bl_ML2_GBReg)
    ef.add_objective(objective_functions.L2_reg)
    ef.max_sharpe()
    weights_ML2_GBReg = ef.clean_weights()
    # weights_ML2_GBReg
    weights_ML2_GBReg_df = pd.DataFrame(weights_ML2_GBReg, index=[today])
    # print(weights_ML2_GBReg_df)
    # weights_ML2_GBReg_df_out = weights_ML2_GBReg_df_out.append(weights_ML2_GBReg_df)
    weights_ML2_GBReg_df.to_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/weights_ML2_GBReg_df.csv')
    #   ---------------- ADDED FOR REPORT and STACKING UP -----------------------------
    w_hist = pd.read_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/w_hist.csv', index_col='Date')
    df = pd.read_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/df.csv', index_col='Date')
    df_ret_calc = df.add_prefix('ret_')
    df_ret_calc.index = pd.to_datetime(df_ret_calc.index).strftime('%m/%d/%Y')
    # df_ret_calc
    w_hist_calc = w_hist.add_prefix('w_')
    w_hist_calc.index = pd.to_datetime(w_hist_calc.index).strftime('%m/%d/%Y')
    port = w_hist_calc.merge(df_ret_calc, how='inner', left_index=True, right_index=True)
    stock_list = tickers_pull
    for stock_str in stock_list:
        port['x_'+stock_str] = port['w_'+stock_str]*port['ret_'+stock_str]
        port = port.drop('w_'+stock_str, axis=1)
        port = port.drop('ret_'+stock_str, axis=1)
    port_wavg_ret = port.sum(axis=1)
    sharpe_ratio_port = port_wavg_ret.mean()/port_wavg_ret.std()
    rm_ret = market_prices_BL_raw.pct_change()
    rm_ret = rm_ret.dropna()
    rm_ret.index = pd.to_datetime(rm_ret.index).strftime('%m/%d/%Y')
    rm = port_wavg_ret.to_frame().join(rm_ret.to_frame())
    sharp_ratio_rm = rm['Adj Close'].mean()/rm['Adj Close'].std()
    sharpe_summary_list = [[ port_wavg_ret.index.max(), sharpe_ratio_port, sharp_ratio_rm]]
    sharpe_summary = pd.DataFrame(sharpe_summary_list, columns=['Date', 'Sharpe Ratio Portfolio (30 days)', 'Sharpe Ratio S&P (30 days)'])
    sharpe_summary_csv = sharpe_summary.set_index('Date')
    sharpe_summary_csv.to_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/sharpe_summary_csv.csv', index_label='Date')
    return_summary_list = [[ port_wavg_ret.index.max(), port_wavg_ret.mean(), port_wavg_ret.std(), rm['Adj Close'].mean(), rm['Adj Close'].std()]]
    return_summary = pd.DataFrame(return_summary_list, columns=['Date', 'Portfolio Avg Return (30 days)', 'Portfolio Return Std (30 days)', 'S&P Avg Return  (30 days)', 'S&P Return Std (30 days)'])
    return_summary_csv = return_summary.set_index('Date')
    return_summary_csv.to_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/return_summary_csv.csv', index_label='Date')
    print("return_summary_csv")
    print(return_summary_csv)
    # ---- finally we can stack up the weights
    weights_ML2_GBReg_df.index.name='Date'
    weights_ML2_GBReg_df.index = pd.to_datetime(weights_ML2_GBReg_df.index).strftime('%m/%d/%Y')
    # w_hist_app = w_hist.append(weights_ML2_GBReg_df)
    print(weights_ML2_GBReg_df)
    w_hist_app = pd.concat([w_hist, weights_ML2_GBReg_df])
    w_hist = w_hist_app.tail(30)
    w_hist.to_csv('C:/Users/Ed/EECS6893_BigDataAnalytics_FALL_2023/proj_fall2023/data_bucket/manual_test_dec12/w_hist.csv', index_label='Date')

In [ ]:
# ----- Run history -------------
from datetime import datetime
for day_str in date_list_x2:
    yfin_end_date = datetime.strptime(day_str[0], '%Y-%m-%d').date()
    today = datetime.strptime(day_str[1], '%m/%d/%Y').date()
    yesterday_date = datetime.strptime(day_str[2], '%m/%d/%Y').date()
    step1_pull_stock_data(yfin_end_date)
    step2_ml_xg()
    print(today)
    step3_news_feed(yesterday_date)
    step4_bl_weight()
    